# Data Preprocessing

對新聞、討論的文章進行資料前處理，包含斷詞、去除標點符號、計算 tf-idf、特徵選取等等，以利後續的模型訓練。

## 使用 CKIP Tagger 進行斷詞

In [1]:
# 先把我們需要的函數載入
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
ws = WS("./data_ckip") # 斷詞
pos = POS("./data_ckip") # 詞性標注
ner = NER("./data_ckip") # 命名實體識別 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
2024-04-04 15:03:33.944737: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ckiptagger/model_ner.py:57: UserWarning: `tf.nn.rnn_c

In [2]:
# 載入資料集
import pandas as pd
data_dir = '../data/'

df_news = pd.read_csv(data_dir + 'news_filtered.csv')

# 提取正文內容
corpus = df_news['content'].tolist()

In [3]:
# 進行斷詞
word_sentence_list = ws(corpus, 
                        sentence_segmentation=True,
                        segment_delimiter_set={'?', '？', '!', '！', '。', ',','，', ';', ':', '、', ' ', '.'})
# 移除標點符號
punc = ['，', '。', '、', '：', '；', '？', '！', '「', '」', '（', '）', '『', '』', '—', '－', '～', '…', '‧', '《', '》', '〈', '〉', '﹏﹏']
eng_punc = [',', '.', ':', ';', '?', '!', '(', ')', '[', ']', '&', '@', '#', '$', '%', '-', '_', '*', '/', '\\', '+', '=', '>', '<', '"', "'", '’', '‘', '“', '”', ' ']

word_sentence_list = [[word for word in sentence if word not in punc] for sentence in word_sentence_list]
word_sentence_list = [[word for word in sentence if word not in eng_punc] for sentence in word_sentence_list]

## 取得 n-gram (n = 1 ~ 3) 的 BOW Count
最後會被存在名為 `bow_count` 的 scipy.sparse._csr.csr_matrix 物件中。

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(ngram_range=(1, 3))
bow_count = bow.fit_transform([' '.join(sentence) for sentence in word_sentence_list])
print(bow_count.shape)

(8617, 1656993)


## 取得 n-gram (n = 1 ~ 3) 的 tf-idf
最後會被存在名為 `tfidf` 的 scipy.sparse._csr.csr_matrix 物件中。

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tv = TfidfVectorizer(ngram_range=(1, 3))
tfidf = tv.fit_transform([' '.join(sentence) for sentence in word_sentence_list])
print(tfidf.shape)

(8617, 1656993)


## Chi-square 特徵選取
根據標籤內容，計算每個詞的 chi-square 值，並選取前 1000 個詞作為特徵。

In [16]:
from sklearn.feature_selection import SelectKBest, chi2
import numpy as np

# 需要預先給定的標籤 Y，我們在這邊暫且先把 Y 隨機分配為 0 或 1
Y = np.random.randint(2, size=(len(corpus),)) # 這是假資料！

# 選取前 1000 個特徵
ch2 = SelectKBest(chi2, k=1000)
X_chi2 = ch2.fit_transform(bow_count, Y)
print(X_chi2.shape)


(8617, 1000)


In [21]:
# 顯示 X_chi2 的特徵名稱
feature_names = bow.get_feature_names_out()
mask = ch2.get_support() # 這個 mask 會告訴你哪些特徵被選取了
selected_features = feature_names[mask]
# 從 selected features 中隨機選取 10 個特徵
print(np.random.choice(selected_features, 10))

['現金 方式' '嚴重' '學會' '投資 等級' '航班 編號 it' '修訂案 討論 公司' '束帶' '事業群' '電站' '合約 金額']


## 儲存處理好的資料

In [22]:
# Save X_chi2
from scipy import sparse
sparse.save_npz(data_dir + 'X_chi2.npz', X_chi2)